In [ ]:
X=data.drop('total_claim_amount', axis=1)
y=data.total_claim_amount

# we drop some useless columns
X=X.drop(columns=['customer', 'effective_to_date'])
Normalize (numerical).
# first numerical and categorical split

X_num=X._get_numeric_data()
X_cat=X.drop(columns=X_num.columns)

print (X.shape, X_num.shape, X_cat.shape)  # check shape
# numerical
X_num.describe()
# normalize (standard)
from sklearn.preprocessing import StandardScaler

X_scaled=StandardScaler().fit_transform(X_num)

def normalize(X):
    X_mean=X.mean(axis=0)
    X_std=X.std(axis=0)
    X_std[X_std==0]=1.0
    X=(X-X_mean)/X_std
    return X

X_num=normalize(X_num)
X_num.head()
(X_num-X_scaled).sum().sum()  # check difference
One Hot/Label Encoding (categorical).
# get dummies pandas
pd.get_dummies(X_cat, drop_first=True).head()
# sklearn

from sklearn.preprocessing import OneHotEncoder

pd.DataFrame(OneHotEncoder(drop='first').fit_transform(X_cat).toarray(),
             columns=OneHotEncoder(drop='first').fit(X_cat).get_feature_names(input_features=X_cat.columns)).head()
Concat DataFrames
X_cat=pd.get_dummies(X_cat, drop_first=True)
X=pd.concat([X_num, X_cat], axis=1) # concat numerical and categorical transformations
X.head()
Linear Regression
Solution:

train-test split.
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test=tts(X, y, test_size=.2)
Apply linear regression.
# statsmodels version

import warnings
warnings.simplefilter('ignore')

import statsmodels.api as sm
from statsmodels.formula.api import ols

X_train_wi=sm.add_constant(X_train)
model=sm.OLS(y_train,X_train_wi).fit()

print(model.summary())
# sklearn version
from sklearn.linear_model import LinearRegression as LinReg

linreg=LinReg()    # model
linreg.fit(X_train, y_train)   # model train
y_pred_linreg=linreg.predict(X_test)   # model prediction
Model Validation
Solution:

R2.
print ('train R2: {} -- test R2: {}'.format(linreg.score(X_train, y_train),
                                            linreg.score(X_test, y_test)))
MSE.
from sklearn.metrics import mean_squared_error as mse


train_mse=mse(linreg.predict(X_train), y_train)
test_mse=mse(linreg.predict(X_test), y_test)

print ('train MSE: {} -- test MSE: {}'.format(train_mse, test_mse))
RMSE.
print ('train RMSE: {} -- test RMSE: {}'.format(train_mse**.5, test_mse**.5))
MAE.
from sklearn.metrics import mean_absolute_error as mae

train_mae=mae(linreg.predict(X_train), y_train)
test_mae=mae(linreg.predict(X_test), y_test)

print ('train MAE: {} -- test MAE: {}'.format(train_mse, test_mse))